# Datasets

In [1]:
# fix relative import
import os, sys
dir2 = os.path.abspath('')
dir1 = os.path.dirname(dir2)
if not dir1 in sys.path: sys.path.append(dir1)

In [2]:
# import libs + list audio backend
from tqdm import tqdm
import torch
import torchaudio
from concurrent.futures import ThreadPoolExecutor, as_completed
print(f"audio backend list: {str(torchaudio.list_audio_backends())}")

audio backend list: ['soundfile']


In [3]:
# dataset scan function with multithreading
def dataset_scan(data, desc, max_workers=4):
    error = False
    
    def process_item(i):
        try:
            audio = data[i][0].shape
            label = data[i][1].shape
        except Exception as e:
            return (i, data.files[i], str(e))
        return None
    
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = {executor.submit(process_item, i): i for i in range(len(data))}
        
        for future in tqdm(as_completed(futures), total=len(data), desc=desc):
            result = future.result()
            if result is not None:
                error = True
                i, file, exception = result
                print(f"Error id: {i} file: {file} - Exception: {exception}")
    
    return error

In [ ]:
# Import: MIR-1K
from crepe.dataset import MIR1KDataset
mir_1k = MIR1KDataset(root_dir=os.path.join(dir1, "dataset/MIR-1K"))

# Test: MIR-1K
print("** mir_1k **")
print(f"lenght of dataset: {len(mir_1k)}")
idx = int(torch.randint(0, int(len(mir_1k)), (1,)))
print(f"random sample id: {int(idx)}")
print(f"id_{idx} audio size: {mir_1k[idx][0].shape}")
print(f"id_{idx} label size: {mir_1k[idx][1].shape}")

error = dataset_scan(mir_1k, "test mir_1k")

if not error:
    print("Dataset analyzed and ready to be used")
if error:
    print("it seams that your dataset is not well formated.")

In [ ]:
# Import: Bach-10
from crepe.dataset import Back10Dataset
back10 = Back10Dataset(root_dir=os.path.join(dir1, "dataset/Bach10"))

# Test: MIR-1K
print("** back10 **")
print(f"lenght of dataset: {len(back10)}")
idx = int(torch.randint(0, int(len(back10)), (1,)))
print(f"random sample id: {int(idx)}")
print(f"id_{idx} audio size: {back10[idx][0].shape}")
print(f"id_{idx} label size: {back10[idx][1].shape}")

error = dataset_scan(back10, "test back10")

if not error:
    print("Dataset analyzed and ready to be used")
if error:
    print("it seams that your dataset is not well formated.")

In [4]:
# Import: Nsynth
from crepe.dataset import NSynthDataset
nsynth = NSynthDataset(root_dir=os.path.join(dir1, "dataset/Nsynth-mixed"), n_samples=30)

# Test: MIR-1K
print("** nsynth **")
print(f"lenght of dataset: {len(nsynth)}")
idx = int(torch.randint(0, int(len(nsynth)), (1,)))
print(f"random sample id: {int(idx)}")
print(f"id_{idx} audio size: {nsynth[idx][0].shape}")
print(f"id_{idx} label size: {nsynth[idx][1].shape}")

error = dataset_scan(nsynth, "test nsynth")

if not error:
    print("Dataset analyzed and ready to be used")
if error:
    print("it seams that your dataset is not well formated.")

** nsynth **
lenght of dataset: 10062
random sample id: 5246
id_5246 audio size: torch.Size([1920000])
id_5246 label size: torch.Size([48000])


test nsynth: 100%|██████████| 10062/10062 [1:49:47<00:00,  1.53it/s]    

Dataset analyzed and ready to be used
